In [1]:
from tqdm import tqdm, trange
import json
from collections import namedtuple, defaultdict
import pandas as pd
import numpy as np
import torch
import os
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.models.word2vec import Word2Vec
import random
from scipy.sparse import csr_matrix
import dgl

import pickle as pkl
import sys
sys.path.append('/mnt/nfs/zhangtl/utils')
from util import myout

Using backend: pytorch


## load

In [2]:
dataset = 'dblp'
df = pd.read_csv(f'../../../01_process/data/{dataset}/Papers.csv') 
df.dropna(subset=['p_id', 'year'], inplace=True)
df.fillna({'refs': '', 'a_ids': '', 'v_id': '', 'fids': '', 'kids': ''}, inplace=True)
df

/home/ztl/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,title,p_id,year,ncites,refs,a_ids,v_id,fids,kids,abst
0,Kiri Jacob Emile Pereire'le,56d85b67dabfae2eee51d618,1800.0,0.0,"53e9b923b7602d970450d298,5736970a6e3b12023e604...",,,"1635,1636,13,1630,8073,95,677,10958,5933,25081,52",6891771,NaN
1,"[Les Désastres de la guerre, pl. 56 : Al cemen...",56d851a9dabfae2eee07e585,1815.0,0.0,53e9ab89b7602d97035361c8,,,"29026,29081,12384,13,654,1482,20,3205",,NaN
2,"Gazeta Warszawska. 1822, nr 24 + dod.",56d88f76dabfae2eeedb33e2,1822.0,0.0,"558c1c7184ae6766fdf11d9f,53e9bd23b7602d97049ae...",,,"8745,12737,2457,298,67854,28,4634,1121,120","30078,1494,17269,1771937,4098,36498,1599",NaN
3,Urkunden und Abhandlungen zur Geschichte des N...,56d85754dabfae2eee32f23c,1824.0,0.0,"53e99e3eb7602d9702703684,53e9acd3b7602d97036b1...",,,"11159,1181,5845,1068,13714,1067,11085,313,2210...",,NaN
4,"Kurjer Warszawski. 1831, nr 114",56d83ebedabfae2eee76ccac,1831.0,0.0,"53e9bc05b7602d97048649c8,53e9aefcb7602d970392e...",,,"17695,15516,660,11384,1293,1014,33925,1687,78,87","30078,36498,1771937,17269,4098,1599,1494",NaN
...,...,...,...,...,...,...,...,...,...,...
5353317,“Did you know this camera tracks your mood?”: ...,603877169e795ea1fb778245,2021.0,0.0,,"53f4e3f5dabfaee4dc8b856b,53f4ae2edabfaedd74eb9...",,,,NaN
5353318,“Did you know this camera tracks your mood?”: ...,603877169e795ea1fb778246,2021.0,0.0,,"53f4ae2edabfaedd74eb918f,,",,,,NaN
5353319,“Relationship Between Learning by Teaching wit...,60703a26e4510cd7c8b7ffb7,2021.0,-1.0,,,,,"8998,1153,1043,2478,1208885,148596,1936",Learning through teaching robots has been show...
5353320,ℓp Subspace Embedding in Input Sparsity Time.,6008039a91e011f078795c97,2021.0,-1.0,"53e9a7c1b7602d97031002a4,53e9b20cb7602d9703ca7...",,,"1976,1824,28,120",,NaN


In [3]:
start_year, end_year = 2000, 2022
drop_id = df[(df.year < start_year) | (df.year >= end_year)].index
print(f'Drop {len(drop_id)} rows') # 559749
papers = df.drop(drop_id) # 3547589
papers

Drop 655057 rows


,title,p_id,year,ncites,refs,a_ids,v_id,fids,kids,abst
655058,Human-Robot Interaction through Spoken Langua...,53e9b4f4b7602d970401fd57,2000.0,23.0,"53e9bc42b7602d97048b5a74,53e9b043b7602d9703aa6...",53f4c7e7dabfaee57d77ca04,555037837cea80f95418b43e,"1518,13,2001,5922,277,20,496,674,4117,1046,3357",,NaN
655059,The OCoN Approach for Object-Oriented Distrib...,53e9985fb7602d9702097c47,2000.0,14.0,"53e9a0d9b7602d97029c1344,53e9b477b7602d9703f7a...","53f43468dabfaee2a1cd3660,5405fab6dabfae91d302341b",555036d27cea80f95415a4df,"2608,2646,13,19206,595,126,127,130,596,52","1956,3197,46411,11475,52877,46452,11214",There are many significant problems related t...
655060,Visual Modeling of Object-oriented Distribute...,53e998e1b7602d970211fed9,2000.0,13.0,"53e9a432b7602d9702d4d588,558ab1cfe4b037c087589...","53f43468dabfaee2a1cd3660,5405fab6dabfae91d302341b",,"7285,7511,38,3220,15260,13,15267,2795,7838,17446",305201,Software engineering for distributed systems i...
655061,"""...but can you prove it?""",53e9aa3ab7602d97033a5a30,2000.0,0.0,,53f397e7dabfae4b34a7e922,555037677cea80f954185b1e,"1565,33,618,20333,13,65375,1358,902,12570,28846","641199,1059743,10909,2936917,2936918,10920,487...",Technical writers are called upon for many dif...
655062,"""A framework for system specification using ch...",53e9b0bcb7602d9703b30ee8,2000.0,0.0,,,539ffba2831432abcb5eeb7f,,,NaN
...,...,...,...,...,...,...,...,...,...,...
5353317,“Did you know this camera tracks your mood?”: ...,603877169e795ea1fb778245,2021.0,0.0,,"53f4e3f5dabfaee4dc8b856b,53f4ae2edabfaedd74eb9...",,,,NaN
5353318,“Did you know this camera tracks your mood?”: ...,603877169e795ea1fb778246,2021.0,0.0,,"53f4ae2edabfaedd74eb918f,,",,,,NaN
5353319,“Relationship Between Learning by Teaching wit...,60703a26e4510cd7c8b7ffb7,2021.0,-1.0,,,,,"8998,1153,1043,2478,1208885,148596,1936",Learning through teaching robots has been show...
5353320,ℓp Subspace Embedding in Input Sparsity Time.,6008039a91e011f078795c97,2021.0,-1.0,"53e9a7c1b7602d97031002a4,53e9b20cb7602d9703ca7...",,,"1976,1824,28,120",,NaN


## build graph

In [5]:
dataset = 'dblp'
p_id2emb = pkl.load(open(f'../../../01_process/data_papers/{dataset}/p_id2emb.pkl', 'rb'))
myout(p_id2emb)
def update_idx(idx, dic, cnt, feats, rel, feat_dim, no_emb):
    if idx not in dic:
        dic[idx] = cnt
        cnt += 1
        if rel == 0:
            try:
                feats.append(torch.from_numpy(p_id2emb[idx]).to(torch.float32))
            except:
                feats.append(torch.rand(feat_dim).to(torch.float32)-1)
                no_emb += 1
        elif rel == 1:
            feats.append(torch.rand(feat_dim).to(torch.float32)+1)
        elif rel == 2:
            feats.append(torch.rand(feat_dim).to(torch.float32)+2)
        elif rel == 3:
            feats.append(torch.rand(feat_dim).to(torch.float32))
    return dic, cnt, feats, no_emb

p_id2emb : len=5353302, dict([56d85b67dabfae2eee51d618: [-0.14996171 -0.24289319 -0.20069335  0.34483987  0.13096131 -0.17293683
  0.2025778   0.17115092  0.10038076 -0.18821181 -0.03741079  0.07061126
  0.40238005  0.3366903  -0.2504851  -0.32120442  0.09956586 -0.08026345
 -0.39538717  0.02969697  0.05036463  0.16746783 -0.22825173  0.38427606
 -0.18321097 -0.08896309 -0.06993408 -0.14664915  0.2746881   0.21373355
  0.06928721 -0.06848475  0.43111807  0.09308034  0.30405688 -0.11347856
 -0.1918604   0.13551643 -0.22350234  0.01857326 -0.0768945  -0.41133887
 -0.40798545 -0.09700473  0.14057675  0.17822282  0.15218355 -0.14711323
  0.00413425 -0.00090013 -0.03764547  0.06710418  0.00925352  0.09667202
 -0.07114001 -0.09749309 -0.19448593  0.05420415  0.29595003 -0.35120714
 -0.41959375 -0.02350043 -0.01356852  0.12763533 -0.10448362  0.14822769
 -0.4964648  -0.06091559 -0.24984936  0.3266589   0.04997223  0.27689603
 -0.14685804 -0.14400679 -0.00903094  0.1872166   0.03084855 -0.1297

In [7]:
start_year, end_year = 2000, 2022
feat_dim = 128

id2nid, cnt, no_emb = {}, 0, 0
lst, feats = [], []
for i in trange(len(papers)):
    ts = int(papers['year'].iloc[i])
    p_id = str(papers['p_id'].iloc[i])
    refs = papers['refs'].iloc[i]
    
    id2nid, cnt, feats, no_emb = update_idx(p_id, id2nid, cnt, feats, 0, feat_dim, no_emb)
    if len(refs)>0:
        rlst = refs.split(',')
        for ref in rlst:
            if ref != '':
                ref = str(ref)
                id2nid, cnt, feats, no_emb = update_idx(ref, id2nid, cnt, feats, 0, feat_dim, no_emb)
                lst.append((id2nid[p_id], id2nid[ref], 0, ts))
        
feat = torch.stack(feats)
src = torch.tensor([item[0] for item in lst])
tgt = torch.tensor([item[1] for item in lst])
rel = torch.tensor([item[2] for item in lst])
tsp = torch.tensor([item[3] for item in lst])
myout(feat, src, tgt, rel, tsp)

100%|██████████| 4698245/4698245 [04:01<00:00, 19427.58it/s]


feat : shape=torch.Size([5029342, 128])
tensor([[-0.3387, -0.2144, -0.2311,  ..., -0.0798,  0.3071, -0.1553],
        [-0.0943, -0.0019,  0.1353,  ..., -0.0379, -0.0962, -0.2468],
        [-0.2017,  0.0200,  0.0147,  ...,  0.0431, -0.0219, -0.2400],
        ...,
        [-0.1403, -0.0404,  0.1263,  ..., -0.0705,  0.0128, -0.2075],
        [-0.0972, -0.3329,  0.0146,  ...,  0.1172,  0.0327, -0.1143],
        [ 0.0457, -0.0281,  0.1176,  ...,  0.0110, -0.0714, -0.2424]])
src : shape=torch.Size([45281685]), tensor([      0,       0,       0,  ..., 5029340, 5029340, 5029340])
tgt : shape=torch.Size([45281685]), tensor([      1,       2,       3,  ..., 1471200, 1212070, 3161810])
rel : shape=torch.Size([45281685]), tensor([0, 0, 0,  ..., 0, 0, 0])
tsp : shape=torch.Size([45281685]), tensor([2000, 2000, 2000,  ..., 2021, 2021, 2021])


In [9]:
graph = dgl.graph((src, tgt), num_nodes=len(feat))
graph.ndata['feat'] = feat

nid2id = {vv: kk for kk, vv in id2nid.items()}
graph.ndata['raw_nid'] = torch.arange(len(feat))

graph.edata['rel'] = rel
graph.edata['ts'] = tsp
graph

Graph(num_nodes=5029342, num_edges=45281685,
      ndata_schemes={'feat': Scheme(shape=(128,), dtype=torch.float32), 'raw_nid': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'rel': Scheme(shape=(), dtype=torch.int64), 'ts': Scheme(shape=(), dtype=torch.int64)})

In [10]:
dgl.save_graphs(f'../data/{dataset}/graph.bin', [graph])

In [11]:
json.dump(id2nid, open(f'../data/{dataset}/id2nid.json', 'w'))

## gen cites

In [13]:
cites = {}
print(start_year, end_year)
for year in range(start_year, end_year):
    cites[year] = defaultdict(int)

papers.fillna({'refs': ''}, inplace=True)
for i in trange(len(papers)):
    year = int(papers['year'].iloc[i])
    refs = papers['refs'].iloc[i]
    if len(refs)>0:
        rlst = refs.split(',')
        rlst = [str(item) for item in rlst]
        for ref in rlst:
            if ref != '':
                cites[year][ref] += 1

2000 2022


100%|██████████| 4698245/4698245 [01:49<00:00, 43066.28it/s]


In [15]:
tsp = graph.edata['ts']
ts_vals, ts_cuts = np.unique(tsp.numpy(), return_index=True)
ts_cuts = list(ts_cuts) + [len(tsp.numpy())]

num_ts = len(ts_vals)
ts_infos = np.stack([ts_vals, ts_cuts[0:num_ts], ts_cuts[1:num_ts+1]]).transpose()
myout(ts_cuts, ts_vals, ts_infos)

ts_cuts : len=23, list([0, 438603, 922575, ..., 41147395, 45096993, 45281685])
ts_vals : shape=(22,), [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021]
ts_infos : shape=(22, 3)
[[    2000        0   438603]
 [    2001   438603   922575]
 [    2002   922575  1484950]
 [    2003  1484950  2166108]
 [    2004  2166108  3029356]
 [    2005  3029356  4084927]
 [    2006  4084927  5373080]
 [    2007  5373080  6823703]
 [    2008  6823703  8446545]
 [    2009  8446545 10272578]
 [    2010 10272578 12284389]
 [    2011 12284389 14463021]
 [    2012 14463021 16902001]
 [    2013 16902001 19700605]
 [    2014 19700605 22832745]
 [    2015 22832745 26319362]
 [    2016 26319362 29578712]
 [    2017 29578712 33228586]
 [    2018 33228586 37355494]
 [    2019 37355494 41147395]
 [    2020 41147395 45096993]
 [    2021 45096993 45281685]]


In [16]:
labels = {}
nid2id = {v:k for k,v in id2nid.items()}
for year in range(start_year, end_year):
    left, right = ts_infos[np.where(ts_infos[:, 0]==year)[0][0], 1:]
    nids = graph.edges()[0][left:right].unique().tolist()
    ids = [nid2id[nid] for nid in nids]
    
    pdf = pd.DataFrame({'id': ids, 'nid': nids})
    tbar = trange(year+1, end_year, desc=str(year))
    for yy in tbar:
        cdf = pd.DataFrame({'id': list(cites[yy].keys()), str(yy): list(cites[yy].values())})
        cdf[str(yy)] = cdf[str(yy)].astype('float32')
        
        pdf = pd.merge(pdf, cdf, how='left', on='id')
        tbar.set_postfix(year=year, pdf=len(pdf))
    pdf.fillna(0, inplace=True)
    labels[year] = pdf
    

2020: 100%|██████████| 1/1 [00:00<00:00,  5.69it/s, pdf=221945, year=2020]
2021: 0it [00:00, ?it/s]


In [17]:
labels[2005]

,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,53e99d96b7602d9702653bcf,7284,16.0,11.0,24.0,14.0,16.0,16.0,11.0,8.0,10.0,10.0,10.0,8.0,8.0,4.0,3.0,0.0
1,53e997f1b7602d9701ff150d,8624,31.0,33.0,42.0,46.0,35.0,40.0,51.0,40.0,10.0,12.0,10.0,4.0,5.0,5.0,2.0,0.0
2,573695436e3b12023e471379,11981,3.0,3.0,3.0,7.0,0.0,1.0,4.0,9.0,7.0,2.0,1.0,1.0,1.0,2.0,2.0,0.0
3,53e9bd82b7602d9704a28373,15759,16.0,22.0,21.0,18.0,13.0,15.0,7.0,8.0,9.0,11.0,10.0,8.0,4.0,4.0,1.0,0.0
4,53e9a501b7602d9702e21b51,16419,8.0,9.0,10.0,7.0,11.0,7.0,4.0,5.0,4.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115315,56d8b36edabfae2eeef4b50a,923848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115316,5ce2ba3bced107d4c6fb69e4,923859,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115317,56d8b0c5dabfae2eeedf4d34,923860,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115318,56d81595dabfae2eee6f6096,923869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
pkl.dump(labels, open(f'../data/{dataset}/labels.pkl', 'wb'))

## cum log labels

In [19]:
def cumulative_log(df):
    colsn = list(df.columns)
    for i in range(3, len(colsn)):
        df[colsn[i]] += df[colsn[i-1]]
    df.iloc[:, 2:] = np.log(df.iloc[:, 2:] + 1)
    return df

labels_cum_log = {}
for year in range(start_year, end_year-2): # 2000, 2020
    labels_cum_log[year] = cumulative_log(labels[year])
labels_cum_log[end_year-2] = labels[end_year-2]
print(len(labels_cum_log))
labels_cum_log[2005]

21


,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,53e99d96b7602d9702653bcf,7284,2.833213,3.332205,3.951244,4.189655,4.406719,4.584968,4.691348,4.762174,4.844187,4.919981,4.990433,5.043425,5.093750,5.117994,5.135798,5.135798
1,53e997f1b7602d9701ff150d,8624,3.465736,4.174387,4.672829,5.030438,5.236442,5.429346,5.631212,5.765191,5.796058,5.831882,5.860786,5.872118,5.886104,5.899898,5.905362,5.905362
2,573695436e3b12023e471379,11981,1.386294,1.945910,2.302585,2.833213,2.833213,2.890372,3.091043,3.433987,3.637586,3.688879,3.713572,3.737670,3.761200,3.806663,3.850148,3.850148
3,53e9bd82b7602d9704a28373,15759,2.833213,3.663562,4.094345,4.356709,4.510859,4.663439,4.727388,4.795791,4.867535,4.948760,5.017280,5.068904,5.093750,5.117994,5.123964,5.123964
4,53e9a501b7602d9702e21b51,16419,2.197225,2.890372,3.332205,3.555348,3.828641,3.970292,4.043051,4.127134,4.189655,4.219508,4.234107,4.262680,4.290460,4.290460,4.290460,4.290460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115315,56d8b36edabfae2eeef4b50a,923848,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
115316,5ce2ba3bced107d4c6fb69e4,923859,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
115317,56d8b0c5dabfae2eeedf4d34,923860,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
115318,56d81595dabfae2eee6f6096,923869,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
pkl.dump(labels_cum_log, open(f'../data/{dataset}/labels_cum_log.pkl', 'wb'))